In [ ]:
import numpy as np
import math
import time

# Константы
N = 10**8  # 100 миллионов элементов
print(f"📊 Array size: {N:,} elements")

In [ ]:
def calculate_error(reference, result):
    """Вычисление средней абсолютной ошибки"""
    return np.mean(np.abs(reference - result))


In [ ]:
def test_sin_implementations():
    """Тестирование различных реализаций синуса"""
    print("\n🎯 Calculating reference values...")
    start_time = time.time()

    # Референсные значения с максимальной точностью (float64)
    indices = np.arange(N, dtype=np.float64)
    cpu_reference_double = np.sin((indices % 360) * math.pi / 180.0)
    cpu_reference_float = cpu_reference_double.astype(np.float32)

    ref_time = time.time() - start_time
    print(f"✅ Reference calculation finished in {ref_time*1000:.2f} ms")

    print(f"\n🧪 Testing different implementations:")
    print("=" * 70)
    print(f"{'Method':<25} | {'Error':<15} | {'Time (ms)':<12}")
    print("-" * 70)

    # 1. NumPy double precision (аналог sin(double) в CUDA)
    start_time = time.time()
    result_numpy_double = np.sin((indices % 360) * np.pi / 180.0)
    time_taken = (time.time() - start_time) * 1000
    error = calculate_error(cpu_reference_double, result_numpy_double)
    print(f"{'numpy double':<25} | {error:<15.2e} | {time_taken:>10.2f}")

    # 2. NumPy float32 (аналог sinf(float) в CUDA)
    start_time = time.time()
    indices_float = indices.astype(np.float32)
    result_numpy_float = np.sin((indices_float % 360) * np.float32(math.pi) / 180.0)
    time_taken = (time.time() - start_time) * 1000
    error = calculate_error(cpu_reference_float, result_numpy_float)
    print(f"{'numpy float32':<25} | {error:<15.2e} | {time_taken:>10.2f}")

    # 3. Math.sin с double -> float (аналог sin(double)->float)
    start_time = time.time()
    result_math_double_float = np.array([math.sin((i % 360) * math.pi / 180.0) for i in range(N)], dtype=np.float32)
    time_taken = (time.time() - start_time) * 1000
    error = calculate_error(cpu_reference_float, result_math_double_float)
    print(f"{'math.sin->float32':<25} | {error:<15.2e} | {time_taken:>10.2f}")

    # 4. Math.sin с float вычислениями (точный аналог sinf в CUDA)
    start_time = time.time()
    result_math_float = np.array([math.sin(np.float32((i % 360) * math.pi / 180.0)) for i in range(N)], dtype=np.float32)
    time_taken = (time.time() - start_time) * 1000
    error = calculate_error(cpu_reference_float, result_math_float)
    print(f"{'math.sin(float)':<25} | {error:<15.2e} | {time_taken:>10.2f}")

    return cpu_reference_double, cpu_reference_float


In [ ]:
def analyze_precision_detailed():
    """Детальный анализ точности для разных углов"""
    print("\n🔬 Detailed Precision Analysis")
    print("=" * 50)

    test_angles = [0, 30, 45, 60, 90, 180, 270, 359]

    print(f"\n{'Angle':>6} | {'Exact (double)':>15} | {'Float32':>15} | {'Error':>15}")
    print("-" * 70)

    for angle in test_angles:
        exact_rad = angle * math.pi / 180.0
        exact_sin = math.sin(exact_rad)

        angle_float32 = np.float32(angle * math.pi / 180.0)
        sin_float32 = math.sin(angle_float32)

        error = abs(exact_sin - sin_float32)
        print(f"{angle:6}° | {exact_sin:15.10f} | {sin_float32:15.10f} | {error:15.2e}")

In [ ]:
def analyze_error_sources():
    """Анализ источников ошибок"""
    print("\n📊 Error Sources Analysis")
    print("=" * 40)

    # Анализируем несколько критических точек
    test_points = [0, 359, 360, 719, 720, 1000000, 10000000]

    print(f"\n{'i':>10} | {'i%360':>6} | {'Exact':>12} | {'Float32':>12} | {'Error':>12}")
    print("-" * 65)

    for i in test_points:
        exact_angle = (i % 360) * math.pi / 180.0
        exact_sin = math.sin(exact_angle)

        angle_float32 = np.float32((i % 360) * math.pi / 180.0)
        sin_float32 = math.sin(angle_float32)

        error = abs(exact_sin - sin_float32)
        print(f"{i:10} | {i%360:6} | {exact_sin:12.8f} | {sin_float32:12.8f} | {error:12.2e}")

In [ ]:
def analyze_periodic_effects():
    """Анализ периодических эффектов"""
    print("\n📈 Periodic Effects Analysis")
    print("=" * 40)

    # Анализируем ошибки на границах периодов
    critical_points = [359, 360, 361, 719, 720, 721, 359999, 360000, 360001]

    print(f"\n{'i':>8} | {'i%360':>6} | {'Exact':>12} | {'Float32':>12} | {'Error':>12}")
    print("-" * 65)

    for i in critical_points:
        exact_angle = (i % 360) * math.pi / 180.0
        exact_sin = math.sin(exact_angle)

        angle_float32 = np.float32((i % 360) * math.pi / 180.0)
        sin_float32 = math.sin(angle_float32)

        error = abs(exact_sin - sin_float32)
        print(f"{i:8} | {i%360:6} | {exact_sin:12.8f} | {sin_float32:12.8f} | {error:12.2e}")

In [ ]:
def theoretical_limits():
    """Теоретические пределы точности"""
    print("\n📐 Theoretical Precision Limits")
    print("=" * 40)

    print(f"float32 machine epsilon: {np.finfo(np.float32).eps:.2e}")
    print(f"float64 machine epsilon: {np.finfo(np.float64).eps:.2e}")
    print(f"float32 significant digits: ~{np.finfo(np.float32).precision}")
    print(f"float64 significant digits: ~{np.finfo(np.float64).precision}")

    print(f"\n🎯 Expected error ranges:")
    print(f"float32 trigonometric functions: 1e-7 to 1e-6")
    print(f"float64 trigonometric functions: 1e-16 to 1e-15")


In [11]:
def main():
    """Основная функция"""
    print("🚀 Starting sine functions precision analysis...")
    print("💡 Note: Using CPU implementation as GPU is not available")
    print("   The results are directly comparable to CUDA implementations\n")

    # Запускаем тесты
    ref_double, ref_float = test_sin_implementations()

    # Дополнительный анализ
    analyze_precision_detailed()
    analyze_error_sources()
    analyze_periodic_effects()
    theoretical_limits()


# Запуск
if __name__ == "__main__":
    main()
    print(f"\n🎉 Analysis completed successfully!")
    print("💡 To run with GPU, enable GPU in Colab: Runtime → Change runtime type → GPU")

SINE FUNCTIONS PRECISION ANALYSIS - GOOGLE COLAB
📊 Array size: 100,000,000 elements
🚀 Starting sine functions precision analysis...
💡 Note: Using CPU implementation as GPU is not available
   The results are directly comparable to CUDA implementations


🎯 Calculating reference values...
✅ Reference calculation finished in 6204.92 ms

🧪 Testing different implementations:
Method                    | Error           | Time (ms)   
----------------------------------------------------------------------
numpy double              | 0.00e+00        |    4451.32
numpy float32             | 1.20e-02        |    3522.73
math.sin->float32         | 0.00e+00        |   28724.28
math.sin(float)           | 4.26e-08        |   90504.63

🔬 Detailed Precision Analysis

 Angle |  Exact (double) |         Float32 |           Error
----------------------------------------------------------------------
     0° |    0.0000000000 |    0.0000000000 |        0.00e+00
    30° |    0.5000000000 |    0.5000000126

# 📊 Анализ результатов сравнения реализаций синуса

## 🔍 Основные наблюдения

### 🎯 Сравнение методов

| Метод | Ошибка | Время (мс) | Статус |
|-------|--------|------------|---------|
| `numpy double` | `0.00e+00` | 4451.32 | ✅ **Идеально** |
| `numpy float32` | `1.20e-02` | 3522.73 | ❌ **Аномалия** |
| `math.sin->float32` | `0.00e+00` | 28724.28 | ✅ **Идеально** |
| `math.sin(float)` | `4.26e-08` | 90504.63 | ✅ **Нормально** |

## ⚠️ Критическая проблема

### ❌ Аномалия numpy float32
- **Ошибка**: `1.20e-02` - **на 5 порядков больше ожидаемой!**
- **Ожидалось**: ~1e-7 (соответственно теоретическому пределу float32)
- **Проблема**: Вероятно, ошибка в вычислениях или конвертации типов

## ✅ Нормальные результаты

### 1. Double precision методы
- `numpy double`: `0.00e+00` - идеальная точность
- `math.sin->float32`: `0.00e+00` - идеальная точность вычислений
- Соответствуют теоретическому пределу float64 (`2.22e-16`)

### 2. Float32 точность
- `math.sin(float)`: `4.26e-08` - **нормальная ошибка** для float32
- Соответствует теоретическому пределу float32 (`1.19e-07`)

## 🔬 Детальный анализ точности по углам

| Угол | Точность | Статус |
|------|----------|---------|
| **0°** | `0.00e+00` | ✅ Идеально |
| **30°** | `1.26e-08` | ✅ Отлично |
| **45°** | `1.55e-08` | ✅ Отлично |
| **60°** | `1.46e-08` | ✅ Отлично |
| **90°** | `9.99e-16` | ✅ Практически идеально |
| **180°** | `8.74e-08` | ✅ Хорошо |
| **270°** | `1.11e-16` | ✅ Практически идеально |
| **359°** | `2.03e-07` | ⚠️ Наихудший случай |

## 📈 Производительность

### 🚀 Самые быстрые:
1. **`numpy float32`**: 3522.73 мс (но с аномальной ошибкой)
2. **`numpy double`**: 4451.32 мс

### 🐌 Самые медленные:
1. **`math.sin(float)`**: 90504.63 мс (в 25 раз медленнее NumPy)
2. **`math.sin->float32`**: 28724.28 мс

## 🔍 Анализ периодических эффектов

### 📊 Ключевые наблюдения:
- ✅ **Периодичность 360° работает корректно**
- ✅ **Ошибки стабильны** для одинаковых углов
- ✅ **Накопление ошибок минимально** благодаря периодичности
- ⚠️ **Наибольшие ошибки** на границах периодов (359° → 360°)
